<a href="https://colab.research.google.com/github/CHOEMINGI/Hatrock_HW/blob/main/%ED%86%B5%ED%95%A9%EB%B3%B8_%ED%96%A5%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## 패키지 영구 설치하기

In [104]:
# 다운 받은 패키지 바로 import할 수 있게 하기
sys.path.append('/content/drive/MyDrive/additional_package')

In [105]:
import pandas as pd
from pandas import DataFrame  as df
import konlpy
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [109]:
file_1 = '/content/perfume.csv'
file_2= '/content/perfume_1.csv'

### **통코드**

In [110]:
perfume_df= pd.read_csv("perfume.csv", encoding='cp949')
perfume_df.head(5)

# product= 이름 종류 모음
product=perfume_df.Name.value_counts().index

# 여러 행의 리뷰를 한 리스트로 저장
def review(i): 
  return str(perfume_df[perfume_df['Name']==product[i]].Review.values)

################################################################################################################

#리뷰를 제품별로 하나의 값에 모은 새로운 데이터 프레임 만들기
pf_new_df= df(columns=["Name","Price","document"])

# 가격이 같은 제품들 때문에 아래와 같이 할당

for i in range(len(product)):
  # 각 제품의 가격을 집합으로 중복을 제거하고 가져와서 1대1 대응이 되도록 집합을 다시 리스트로 가져와 전처리
  price= list(set(perfume_df.Price[perfume_df.Name==product[i]]))[0]
  
  # 각 행마다 제품, 가격, 리뷰 가져오기
  pf_new_df.loc[i]=(product[i],price,review(i))

pf_new_df.head(5)

################################################################################################################

import re  # 정규식 표현 모듈 re

# 한글과 띄어쓰기 외 다른 요소 제거 함수 spec_words 생성
def spec_word(s): 
  hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
  result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거 
  return(result)

# 모든 향수에 적용
for i in range(len(product)):
  pf_new_df.document[i]=spec_word(pf_new_df.document[i])

pf_new_df.head(5)

k=0
  ################################################################################################################
for document in pf_new_df.document:

  from konlpy.tag import Okt

  okt = Okt()

  # 형태소
  morph=okt.morphs(document)

  # 구 
  phrase= okt.phrases(document)
  #print(phrase[:100])

  morph= spec_word(str(morph))

  #품사 태깅
  posing= okt.pos(morph)
  posing[:10]

  ################################################################################################################

  # 키워드 추출 시 의미가 없이 많이 사용되는 동사, 조사, 접미사, 접속사 제거
  pos=[]
  for item in posing:
    if "Verb" not in item and "Josa" not in item and "Suffix" not in item and "Conjunction" not in item:
      pos.append(item)
  pos[:10]

  ################################################################################################################

  # 영어와 특수문자 제거하기
  # 정규 표현식 함수 정의

  pos=str(pos)
  import re 
  def spec_word(s): 
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거 
    return(result)

  pos= spec_word(pos)
  pos= pos.split(" ")

  # 형태소에서 한 글자 키워드는 제외하기
  pos= [a for a in pos if len(a) > 1]
  
  ################################################################################################################
  #불용어 제거하기
  #통상적으로 의미를 갖지 않는 불용어들의 사전을 RANKS NL에서 받아서 활용
  stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()

  #향수 리뷰에 자주 등장하지만 의미가 없는 단어들을 추가
  perfume_stopwords=["''", "향수", "제품", "향기", "취향","냄새","너무","같아요","사이즈","생각","느낌","입니다","그냥","같은","''"] 

  stopword= [ i[0] for i in stopwords] +perfume_stopwords



  ################################################################################################################

  # 불용어 제거하기
  from nltk.tokenize import word_tokenize 

  def remove_stopwords(sentence): # 불용어 제거 함수 remove_stopwords
    word_tokens = sentence

    result = [word for word in word_tokens if not word in stopwords]
    return result

  removed_morph=remove_stopwords(pos)
  


  ################################################################################################################

  x_data= removed_morph
  #stopwords = stopword
  #for i, document in enumerate(x_data):
      #clean_words = [] 
      #for word in nltk.tokenize.word_tokenize(document): 
          #if word not in stopwords: #불용어 제거
              #clean_words.append(word)    
      #x_data[i] = ' '.join(clean_words) 
  
  corpus=[]
  corpus.append(x_data)           
 

  ################################################################################################################

  #!pip install Counter
  from collections import Counter

  # 키워드 최다빈도 단어 도출

  print("향수",product[k],"의 핵심 키워드:",Counter(x_data).most_common(20))
  k+=1
  corpus.append(x_data)



향수 페라가모 인칸토 참 EDT 30ml 의 핵심 키워드: [('향수', 141), ('너무', 96), ('좋아요', 77), ('지속', 71), ('구매', 68), ('같아요', 66), ('달달', 56), ('좋은', 52), ('선물', 51), ('여름', 49), ('사용', 48), ('가격', 45), ('ㅎㅎ', 35), ('입니다', 35), ('뿌리', 32), ('제품', 29), ('진짜', 29), ('칸토', 26), ('청포도', 25), ('쓰기', 25)]
향수 랑방 루머 2 로즈 30ml 의 핵심 키워드: [('향수', 120), ('너무', 86), ('좋아요', 65), ('랑방', 55), ('같아요', 49), ('지속', 48), ('구매', 48), ('선물', 44), ('사용', 43), ('향기', 32), ('좋은', 27), ('입니다', 26), ('정말', 24), ('냄새', 23), ('매장', 22), ('달달', 22), ('친구', 21), ('제품', 20), ('잔향', 20), ('ㅎㅎ', 19)]
향수 지미추 블러썸 EDP 40ml 의 핵심 키워드: [('향수', 97), ('달달', 86), ('너무', 79), ('좋아요', 59), ('지속', 47), ('구매', 46), ('선물', 40), ('같아요', 39), ('미추', 29), ('향기', 22), ('좋은', 22), ('생각', 21), ('제품', 21), ('냄새', 20), ('ㅎㅎ', 20), ('뿌리', 19), ('잔향', 18), ('좋아하는', 18), ('사용', 18), ('친구', 17)]
향수 지미추 블러썸 EDP 60ml 의 핵심 키워드: [('너무', 75), ('달달', 74), ('향수', 65), ('좋아요', 55), ('지속', 39), ('구매', 39), ('선물', 38), ('사용', 24), ('입니다', 22), ('미추', 21), ('같아요', 20), ('추천',

### **데이터 불러오기**

In [111]:
perfume_df= pd.read_csv("perfume.csv", encoding='cp949')
perfume_df.head(5)

,Unnamed: 0,Brand,Name,Price,Review
0,0,필로소피,필로소피 어메이징그레이스 매그놀리아 EDT 15ml,"25,000",작은 용량으로 사서 갖고다니면서 뿌리고 있어요 향도 맘에 드네요! 대신 지속력이 오...
1,1,필로소피,필로소피 어메이징그레이스 매그놀리아 EDT 15ml,"25,000",역시 필로소피! 향이 은은하니 좋고 사이즈도 귀여워요~
2,2,필로소피,필로소피 어메이징그레이스 매그놀리아 EDT 15ml,"25,000",향은 좋아요. 다만 지속력이 좀 아쉬울 뿐이지ㅜㅜ
3,3,필로소피,필로소피 어메이징그레이스 매그놀리아 EDT 15ml,"25,000",친구가 올리브영에서 다른 제품을 사야하는게 있다고 하길래 같이 들어간김에 향수 시향...
4,4,필로소피,필로소피 어메이징그레이스 매그놀리아 EDT 15ml,"25,000",기존에 가방 속에 넣어 가지고 다니며 쓰던 향수를 다 써서 새로 향수를 구매해야 할...


In [112]:
# product= 이름 종류 모음
product=perfume_df.Name.value_counts().index
product

# 여러 행의 리뷰를 한 리스트로 저장
def review(i): 
  return str(perfume_df[perfume_df['Name']==product[i]].Review.values)

In [113]:
#리뷰를 제품별로 하나의 값에 모은 새로운 데이터 프레임 만들기
pf_new_df= df(columns=["Name","Price","document"])

# 가격이 같은 제품들 때문에 아래와 같이 할당

for i in range(len(product)):
  # 각 제품의 가격을 집합으로 중복을 제거하고 가져와서 1대1 대응이 되도록 집합을 다시 리스트로 가져와 전처리
  price= list(set(perfume_df.Price[perfume_df.Name==product[i]]))[0]
  
  # 각 행마다 제품, 가격, 리뷰 가져오기
  pf_new_df.loc[i]=(product[i],price,review(i))

pf_new_df.head(5)

,Name,Price,document
0,페라가모 인칸토 참 EDT 30ml,"59,000",['두번째 구매 했어요~ 처음 구매후에 일주일을 매일 사용해봤는데 향의 지속시간도 ...
1,랑방 루머 2 로즈 30ml,"39,900",['랑방 루머 로즈 :) 사실 루머 로즈 남자친구는 이향이 뭔가 남자를 유혹하는...
2,지미추 블러썸 EDP 40ml,"67,000",['아니 진짜 향수가 너무 사고 싶은데 뭐 사야할 지 모르겠어서 고민 오조오백번하고...
3,지미추 블러썸 EDP 60ml,"70,000",['정말 저가 좋아하는 향수에요 꼼꼼하게 포장해주셔서 안심이에용'\n '매장에서 샀...
4,필로소피 어메이징그레이스 발레 로즈 EDT 60ml,"69,000",['씻은 향기ㅋㅋㅋㅋ요론거 좋아하는데 겐조랑 요고랑 고민하다가 매장에서 맡아보구 필...


### 한글과 띄어쓰기 제외 모든 요소 제거

In [114]:
import re  # 정규식 표현 모듈 re

# 한글과 띄어쓰기 외 다른 요소 제거 함수 spec_words 생성
def spec_word(s): 
  hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
  result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거 
  return(result)

# 모든 향수에 적용
for i in range(len(product)):
  pf_new_df.document[i]=spec_word(pf_new_df.document[i])

pf_new_df.head(5)

,Name,Price,document
0,페라가모 인칸토 참 EDT 30ml,"59,000",두번째 구매 했어요 처음 구매후에 일주일을 매일 사용해봤는데 향의 지속시간도 긴편이...
1,랑방 루머 2 로즈 30ml,"39,900",랑방 루머 로즈 사실 루머 로즈 남자친구는 이향이 뭔가 남자를 유혹하는 향이라...
2,지미추 블러썸 EDP 40ml,"67,000",아니 진짜 향수가 너무 사고 싶은데 뭐 사야할 지 모르겠어서 고민 오조오백번하고 올...
3,지미추 블러썸 EDP 60ml,"70,000",정말 저가 좋아하는 향수에요 꼼꼼하게 포장해주셔서 안심이에용 매장에서 샀는데 튼튼해...
4,필로소피 어메이징그레이스 발레 로즈 EDT 60ml,"69,000",씻은 향기ㅋㅋㅋㅋ요론거 좋아하는데 겐조랑 요고랑 고민하다가 매장에서 맡아보구 필로소...


In [115]:
# 리뷰만 할당
docs = pf_new_df['document']

### 키워드의 느낌을 최대한 살려서 가져오기 위해 리뷰 말뭉치를 어간 단위로 분리해준다.

In [116]:
from konlpy.tag import Okt

okt = Okt()

# 형태소
morph=okt.morphs(pf_new_df.document[0])

# 구 
phrase= okt.phrases(pf_new_df.document[0])
#print(phrase[:100])

morph= spec_word(str(morph))

In [117]:
#품사 태깅
posing= okt.pos(morph)
posing[:10]

[('두번째', 'Noun'),
 ('구매', 'Noun'),
 ('했어요', 'Verb'),
 ('처음', 'Noun'),
 ('구', 'Noun'),
 ('매', 'Noun'),
 ('후', 'Noun'),
 ('에', 'Josa'),
 ('일주일', 'Noun'),
 ('을', 'Josa')]

In [118]:
# 키워드 추출 시 의미가 없이 많이 사용되는 동사, 조사, 접미사, 접속사 제거
pos=[]
for item in posing:
  if "Verb" not in item and "Josa" not in item and "Suffix" not in item and "Conjunction" not in item:
    pos.append(item)
pos[:10]

[('두번째', 'Noun'),
 ('구매', 'Noun'),
 ('처음', 'Noun'),
 ('구', 'Noun'),
 ('매', 'Noun'),
 ('후', 'Noun'),
 ('일주일', 'Noun'),
 ('매일', 'Noun'),
 ('사용', 'Noun'),
 ('향', 'Noun')]

In [119]:
# 영어와 특수문자 제거하기
# 정규 표현식 함수 정의

pos=str(pos)
import re 
def spec_word(s): 
  hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
  result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거 
  return(result)

pos= spec_word(pos)
pos= pos.split(" ")

In [120]:
# 형태소에서 한 글자 키워드는 제외하기
pos= [a for a in pos if len(a) > 1]
pos[:10]

['두번째', '구매', '처음', '일주일', '매일', '사용', '시간', '편이', '좋아하는', '여서']

### 불용어 제거하기

한 글자 키워드는 의미를 갖기 어려워서 제외해준다.


통상적으로 의미를 갖지 않는 불용어들의 사전을 RANKS NL에서 받아서 활용

In [121]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()

#향수 리뷰에 자주 등장하지만 의미가 없는 단어들을 추가
perfume_stopwords=["향수", "제품", "향기", "취향","냄새","너무","같아요","사이즈","생각","느낌","입니다"] 

stopword= [ i[0] for i in stopwords] +perfume_stopwords
stopword[:10]

['휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해']

In [127]:
# 불용어 제거하기
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize 

def remove_stopwords(sentence): # 불용어 제거 함수 remove_stopwords
  word_tokens = sentence

  result = [word for word in word_tokens if not word in stopwords]
  return result

removed_morph=remove_stopwords(pos)
removed_morph[:10]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['두번째', '구매', '처음', '일주일', '매일', '사용', '편이', '좋아하는', '여서', '구매']

In [128]:
  x_data= removed_morph  
  corpus=[]
  corpus.append(x_data) 

In [129]:
#!pip install Counter
from collections import Counter

# 키워드 최다빈도 단어 도출
Counter(x_data).most_common(20)

[('좋아요', 77),
 ('지속', 71),
 ('구매', 68),
 ('달달', 56),
 ('좋은', 52),
 ('선물', 51),
 ('여름', 49),
 ('사용', 48),
 ('가격', 45),
 ('ㅎㅎ', 35),
 ('뿌리', 32),
 ('진짜', 29),
 ('칸토', 26),
 ('청포도', 25),
 ('쓰기', 25),
 ('배송', 25),
 ('처음', 24),
 ('기분', 23),
 ('많이', 22),
 ('무난', 22)]

## TF-IDF 

불용어 제거를 통해서도 제거되지 않은 잦게 출현하는 단어들에게 패널티를 주기 위해 TF-IDF를 실행한다.

In [130]:
from math import log 

In [131]:
N = len(docs) # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for x_data in docs:
        df += t in x_data
    return log(N/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [132]:
#TF
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(x_data)):
        t = x_data[j]        
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = x_data)
tf_

,두번째,구매,처음,일주일,매일,사용,편이,좋아하는,여서,구매,있길래,병더,구매,처음,달콤한,지나,자스민,달콤하다고,질리,부담스럽지는,사실,개인,차가,있긴,여성,선물,괜찮을듯,올리브영,기간,구매,저렴하게,구매,있어요,예전,면세점,많이,구매,면세점,구매,올리브영,...,계속,중이,이면,지속,있을텐데,뿌리,정도,안나,가지,작은,용량,도포,사람,기분,좋게,여름,그렇게,빨리,아니지만,처음,좋네요,상쾌한,일찍,살껄,그랬어요,선물,좋고요,저렴한,가격,올리브영,구입,배송,빨리,만족합니다,미니어쳐,좋아요,포장,배송,빠르고,만족합니다
0,2,71,24,1,3,48,12,17,7,71,2,1,71,24,12,10,4,1,6,1,3,4,1,2,9,51,1,9,6,71,17,71,11,6,2,22,71,2,71,9,...,13,6,1,75,1,50,12,2,10,6,9,1,13,24,3,51,8,8,3,24,13,3,1,1,1,51,1,7,45,9,19,25,8,12,1,77,10,25,7,12
1,0,58,18,0,3,44,10,13,2,58,0,0,58,18,2,9,0,0,0,0,5,4,0,0,19,44,0,9,8,58,3,58,14,6,0,14,58,0,58,9,...,7,1,0,49,0,30,10,1,13,2,7,0,8,7,0,15,3,1,3,18,10,0,0,0,0,44,0,4,9,9,14,8,1,13,0,67,6,8,5,13
2,0,55,6,1,1,18,12,21,2,55,0,0,55,6,15,5,0,0,1,0,3,4,0,0,6,40,0,10,2,55,9,55,13,4,0,10,55,0,55,10,...,6,3,0,49,0,27,12,2,8,3,6,0,3,6,0,8,0,1,1,6,4,0,0,0,0,40,0,0,13,10,5,6,1,9,0,59,6,6,2,9
3,1,50,15,0,1,24,6,14,2,50,0,0,50,15,7,3,0,0,3,0,0,1,0,0,7,38,0,6,2,50,8,50,5,3,0,5,50,0,50,6,...,6,1,1,41,0,13,6,1,6,0,2,0,9,12,4,8,1,1,3,15,8,1,1,0,0,38,0,0,8,6,7,5,1,7,0,55,4,5,1,7
4,2,39,8,0,4,20,7,10,0,39,0,0,39,8,1,1,0,0,0,0,2,2,0,0,20,12,0,4,2,39,5,39,5,0,0,11,39,0,39,4,...,3,0,1,41,0,30,13,0,1,2,4,0,0,10,0,6,2,2,1,8,8,0,0,0,0,12,1,2,10,4,7,6,2,3,0,55,3,6,0,3
5,3,56,17,0,4,35,12,10,1,56,1,0,56,17,5,5,0,0,1,0,5,7,0,1,16,28,0,14,2,56,9,56,7,3,0,21,56,0,56,14,...,3,4,2,56,0,25,10,0,2,6,4,0,3,5,4,14,4,5,0,17,8,0,0,0,0,28,1,2,5,14,17,7,5,2,0,44,6,7,0,2
6,0,76,19,0,2,50,6,20,3,76,0,0,76,19,1,8,0,0,5,0,1,6,1,0,5,25,0,17,3,76,15,76,5,10,3,16,76,3,76,17,...,6,0,0,37,0,17,6,0,8,4,3,0,7,13,2,6,5,2,3,19,6,0,0,0,0,25,0,4,11,17,14,12,2,2,0,41,9,12,3,2
7,1,30,2,0,1,38,7,2,2,30,0,0,30,2,3,0,0,0,1,0,1,3,0,1,3,13,1,6,1,30,11,30,5,5,1,5,30,1,30,6,...,0,3,1,34,0,19,1,3,4,1,16,0,3,7,1,23,0,2,4,2,4,1,0,0,1,13,1,10,26,6,14,5,2,4,1,44,8,5,1,4
8,1,47,8,0,2,12,6,8,1,47,1,0,47,8,0,1,0,0,0,0,2,5,0,0,8,17,0,7,0,47,2,47,7,0,0,8,47,0,47,7,...,1,2,1,35,0,26,9,2,6,9,9,0,7,3,0,7,4,1,3,8,0,0,1,0,0,17,1,3,9,7,5,1,1,2,0,31,1,1,0,2
9,2,36,2,0,0,9,0,10,1,36,1,0,36,2,0,2,0,0,1,0,0,0,0,0,6,25,0,2,0,36,3,36,6,2,0,5,36,0,36,2,...,0,2,0,29,0,5,3,0,3,0,3,0,2,1,0,11,1,0,2,2,4,1,0,0,0,25,0,0,3,2,7,5,0,3,0,37,4,5,0,3


In [134]:
# IDF
result = []
for j in range(len(x_data)):
    t = x_data[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = x_data, columns = ["IDF"])
idf_

,IDF
두번째,0.753772
구매,0.000000
처음,0.000000
일주일,1.734601
매일,0.194156
...,...
좋아요,-0.057158
포장,0.060625
배송,0.060625
빠르고,0.435318


In [135]:
#TF-IDF
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(x_data)):
        t = x_data[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = x_data)
#tfidf_.index = pf_new_df.Name
tfidf_

,두번째,구매,처음,일주일,매일,사용,편이,좋아하는,여서,구매,있길래,병더,구매,처음,달콤한,지나,자스민,달콤하다고,질리,부담스럽지는,사실,개인,차가,있긴,여성,선물,괜찮을듯,올리브영,기간,구매,저렴하게,구매,있어요,예전,면세점,많이,구매,면세점,구매,올리브영,...,계속,중이,이면,지속,있을텐데,뿌리,정도,안나,가지,작은,용량,도포,사람,기분,좋게,여름,그렇게,빨리,아니지만,처음,좋네요,상쾌한,일찍,살껄,그랬어요,선물,좋고요,저렴한,가격,올리브영,구입,배송,빨리,만족합니다,미니어쳐,좋아요,포장,배송,빠르고,만족합니다
0,1.507544,0.0,0.0,1.734601,0.582468,0.0,0.727495,0.0,1.877848,0.0,1.774606,2.140066,0.0,0.0,2.329872,2.682640,6.938404,2.140066,2.611908,2.140066,1.044920,1.073056,1.734601,2.893838,0.0,0.0,1.734601,0.0,2.089840,0.0,0.0,0.0,-0.628743,0.750979,2.082908,0.0,0.0,2.082908,0.0,0.0,...,1.627121,2.089840,0.635989,0.0,2.140066,-2.857921,0.0,1.774606,0.0,2.611908,1.126468,2.140066,1.627121,0.0,2.661910,0.0,4.245026,2.146112,1.305954,0.0,0.788120,2.261315,1.223775,1.734601,1.734601,0.0,0.887303,2.438147,0.0,0.0,0.0,1.515616,2.146112,0.0,1.734601,-4.401198,0.606246,1.515616,3.047226,0.0
1,0.000000,0.0,0.0,0.000000,0.582468,0.0,0.606246,0.0,0.536528,0.0,0.000000,0.000000,0.0,0.0,0.388312,2.414376,0.000000,0.000000,0.000000,0.000000,1.741533,1.073056,0.000000,0.000000,0.0,0.0,0.000000,0.0,2.786454,0.0,0.0,0.0,-0.800218,0.750979,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.876142,0.348307,0.000000,0.0,0.000000,-1.714752,0.0,0.887303,0.0,0.870636,0.876142,0.000000,1.001305,0.0,0.000000,0.0,1.591885,0.268264,1.305954,0.0,0.606246,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.393227,0.0,0.0,0.0,0.484997,0.268264,0.0,0.000000,-3.829614,0.363748,0.484997,2.176590,0.0
2,0.000000,0.0,0.0,1.734601,0.194156,0.0,0.727495,0.0,0.536528,0.0,0.000000,0.000000,0.0,0.0,2.912340,1.341320,0.000000,0.000000,0.435318,0.000000,1.044920,1.073056,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.696613,0.0,0.0,0.0,-0.743059,0.500653,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.750979,1.044920,0.000000,0.0,0.000000,-1.543277,0.0,1.774606,0.0,1.305954,0.750979,0.000000,0.375489,0.0,0.000000,0.0,0.000000,0.268264,0.435318,0.0,0.242498,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.363748,0.268264,0.0,0.000000,-3.372346,0.363748,0.363748,0.870636,0.0
3,0.753772,0.0,0.0,0.000000,0.194156,0.0,0.363748,0.0,0.536528,0.0,0.000000,0.000000,0.0,0.0,1.359092,0.804792,0.000000,0.000000,1.305954,0.000000,0.000000,0.268264,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.696613,0.0,0.0,0.0,-0.285792,0.375489,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.750979,0.348307,0.635989,0.0,0.000000,-0.743059,0.0,0.887303,0.0,0.000000,0.250326,0.000000,1.126468,0.0,3.549213,0.0,0.530628,0.268264,1.305954,0.0,0.484997,0.753772,1.223775,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.303123,0.268264,0.0,0.000000,-3.143713,0.242498,0.303123,0.435318,0.0
4,1.507544,0.0,0.0,0.000000,0.776624,0.0,0.424372,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.194156,0.268264,0.000000,0.000000,0.000000,0.000000,0.696613,0.536528,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.696613,0.0,0.0,0.0,-0.285792,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.375489,0.000000,0.635989,0.0,0.000000,-1.714752,0.0,0.000000,0.0,0.870636,0.500653,0.000000,0.000000,0.0,0.000000,0.0,1.061257,0.536528,0.435318,0.0,0.484997,0.000000,0.000000,0.000000,0.000000,0.0,0.887303,0.696613,0.0,0.0,0.0,0.363748,0.536528,0.0,0.000000,-3.143713,0.181874,0.363748,0.000000,0.0
5,2.261315,0.0,0.0,0.000000,0.776624,0.0,0.727495,0.0,0.268264,0.0,0.887303,0.000000,0.0,0.0,0.970780,1.341320,0.000000,0.000000,0.435318,0.000000,1.741533,1.877848,0.000000,1.446919,0.0,0.0,0.000000,0.0,0.696613,0.0,0.0,0.0,-0.400109,0.375489,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.375489,1.393227,1.271978,0.0,0.000000,-1.428960,0.0,0.000000,0.0,2.611908,0.500653,0.000000,0.375489,0.0,3.549213,0.0,2.122513,1.341320,0.000000,0.0,0.484997,0.000000,0.000000,0.000000,0.000000,0.0,0.887303,0.696613,0.0,0.0,0.0,0.424372,1.341320,0.0,0.000000,-2.514970,0.363748,0.424372,0.000000,0.0
6,0.000000,0.0,0.0,0.000000,0.388312,0.0,0.363748,0.0,0.804792,0.0,0.000000,0.000000,0.0,0.0,0.194156,2.146112,0.000000,0.000000,2.176590,0.000000,0.348307,1.609584,1.734601,0.000000,0.0,0.0,0.000000,0.0

In [136]:
data = pd.concat([pf_new_df,tfidf_],axis=1)
data.head(5)

,Name,Price,document,두번째,구매,처음,일주일,매일,사용,편이,좋아하는,여서,구매,있길래,병더,구매,처음,달콤한,지나,자스민,달콤하다고,질리,부담스럽지는,사실,개인,차가,있긴,여성,선물,괜찮을듯,올리브영,기간,구매,저렴하게,구매,있어요,예전,면세점,많이,구매,...,계속,중이,이면,지속,있을텐데,뿌리,정도,안나,가지,작은,용량,도포,사람,기분,좋게,여름,그렇게,빨리,아니지만,처음,좋네요,상쾌한,일찍,살껄,그랬어요,선물,좋고요,저렴한,가격,올리브영,구입,배송,빨리,만족합니다,미니어쳐,좋아요,포장,배송,빠르고,만족합니다
0,페라가모 인칸토 참 EDT 30ml,"59,000",두번째 구매 했어요 처음 구매후에 일주일을 매일 사용해봤는데 향의 지속시간도 긴편이...,1.507544,0.0,0.0,1.734601,0.582468,0.0,0.727495,0.0,1.877848,0.0,1.774606,2.140066,0.0,0.0,2.329872,2.682640,6.938404,2.140066,2.611908,2.140066,1.044920,1.073056,1.734601,2.893838,0.0,0.0,1.734601,0.0,2.089840,0.0,0.0,0.0,-0.628743,0.750979,2.082908,0.0,0.0,...,1.627121,2.089840,0.635989,0.0,2.140066,-2.857921,0.0,1.774606,0.0,2.611908,1.126468,2.140066,1.627121,0.0,2.661910,0.0,4.245026,2.146112,1.305954,0.0,0.788120,2.261315,1.223775,1.734601,1.734601,0.0,0.887303,2.438147,0.0,0.0,0.0,1.515616,2.146112,0.0,1.734601,-4.401198,0.606246,1.515616,3.047226,0.0
1,랑방 루머 2 로즈 30ml,"39,900",랑방 루머 로즈 사실 루머 로즈 남자친구는 이향이 뭔가 남자를 유혹하는 향이라...,0.000000,0.0,0.0,0.000000,0.582468,0.0,0.606246,0.0,0.536528,0.0,0.000000,0.000000,0.0,0.0,0.388312,2.414376,0.000000,0.000000,0.000000,0.000000,1.741533,1.073056,0.000000,0.000000,0.0,0.0,0.000000,0.0,2.786454,0.0,0.0,0.0,-0.800218,0.750979,0.000000,0.0,0.0,...,0.876142,0.348307,0.000000,0.0,0.000000,-1.714752,0.0,0.887303,0.0,0.870636,0.876142,0.000000,1.001305,0.0,0.000000,0.0,1.591885,0.268264,1.305954,0.0,0.606246,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.393227,0.0,0.0,0.0,0.484997,0.268264,0.0,0.000000,-3.829614,0.363748,0.484997,2.176590,0.0
2,지미추 블러썸 EDP 40ml,"67,000",아니 진짜 향수가 너무 사고 싶은데 뭐 사야할 지 모르겠어서 고민 오조오백번하고 올...,0.000000,0.0,0.0,1.734601,0.194156,0.0,0.727495,0.0,0.536528,0.0,0.000000,0.000000,0.0,0.0,2.912340,1.341320,0.000000,0.000000,0.435318,0.000000,1.044920,1.073056,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.696613,0.0,0.0,0.0,-0.743059,0.500653,0.000000,0.0,0.0,...,0.750979,1.044920,0.000000,0.0,0.000000,-1.543277,0.0,1.774606,0.0,1.305954,0.750979,0.000000,0.375489,0.0,0.000000,0.0,0.000000,0.268264,0.435318,0.0,0.242498,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.363748,0.268264,0.0,0.000000,-3.372346,0.363748,0.363748,0.870636,0.0
3,지미추 블러썸 EDP 60ml,"70,000",정말 저가 좋아하는 향수에요 꼼꼼하게 포장해주셔서 안심이에용 매장에서 샀는데 튼튼해...,0.753772,0.0,0.0,0.000000,0.194156,0.0,0.363748,0.0,0.536528,0.0,0.000000,0.000000,0.0,0.0,1.359092,0.804792,0.000000,0.000000,1.305954,0.000000,0.000000,0.268264,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.696613,0.0,0.0,0.0,-0.285792,0.375489,0.000000,0.0,0.0,...,0.750979,0.348307,0.635989,0.0,0.000000,-0.743059,0.0,0.887303,0.0,0.000000,0.250326,0.000000,1.126468,0.0,3.549213,0.0,0.530628,0.268264,1.305954,0.0,0.484997,0.753772,1.223775,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.303123,0.268264,0.0,0.000000,-3.143713,0.242498,0.303123,0.435318,0.0
4,필로소피 어메이징그레이스 발레 로즈 EDT 60ml,"69,000",씻은 향기ㅋㅋㅋㅋ요론거 좋아하는데 겐조랑 요고랑 고민하다가 매장에서 맡아보구 필로소...,1.507544,0.0,0.0,0.000000,0.776624,0.0,0.424372,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.194156,0.268264,0.000000,0.000000,0.000000,0.000000,0.696613,0.536528,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.696613,0.0,0.0,0.0,-0.285792,0.000000,0.000000,0.0,0.0,...,0.375489,0.000000,0.635989,0.0,0.000000,-1.714752,0.0,0.000000,0.0,0.870636,0.500653,0.000000,0.000000,0.0,0.000000,0.0,1.061257,0.536528,0.435318,0.0,0.484997,0.000000,0.000000,0.000000,0.000000,0.0,0.887303,0.696613,0.0,0.0,0.0,0.363748,0.536528,0.0,0.000000,-3.143713,0.181874,0.363748,0.000000,0.0


### **키워드 검색모형**

In [138]:
# 키워드 검색 모형

key = input("찾고싶은 향수의 키워드를 입력하세요: ")
data1 = data.sort_values(by=[key],axis=0,ascending=False)
# 키워드 검색 시 해당 단어를 기준으로 내림차순 정렬
data2 = data1.iloc[:,[0,1,2]]
data2.head(10)
# 검색한 이용지에게 향수이름/가격/리뷰 까지 다 보여주는 것이 좋을 것 같습니다.
# 지금 리뷰를 명사 추출한 것으로 추가했는데, 본래 리뷰 텍스트로 추가하면 더 좋을 것 같습니다.

찾고싶은 향수의 키워드를 입력하세요: 처음


ValueError: ignored

In [139]:
#함수 불러오기
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [140]:
#tf-idf 수행

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['document'])
print(tfidf_matrix.shape)

(17, 11195)


In [141]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [142]:
indices = pd.Series(data.index, index=data['Name']).drop_duplicates()
print(indices.head())

Name
페라가모 인칸토 참 EDT 30ml             0
랑방 루머 2 로즈 30ml                 1
지미추 블러썸 EDP 40ml                2
지미추 블러썸 EDP 60ml                3
필로소피 어메이징그레이스 발레 로즈 EDT 60ml    4
dtype: int64


### **선택한 향수와 가장 유사향수 추천**

In [143]:
def get_recommendations(Name, cosine_sim=cosine_sim):
    # 선택한 향수의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 향수를 가지고 연산할 수 있습니다.
    idx = indices[Name]

    # 모든 향수에 대해서 해당 향수와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 향수들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 5개의 향수를 받아옵니다.
    sim_scores = sim_scores[1:6]

    # 가장 유사한 5개의 향수의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 5개의 향수의 제목을 리턴합니다.
    return data['Name'].iloc[movie_indices]

In [144]:
# 향수 인덱스
idx = id2perfume['페라가모 인칸토 참 EDT 30ml']
# 자기 자신을 제외한 향수들의 유사도 및 인덱스를 추출  
sim_scores = [(i, c) for i, c in enumerate(cosine_sim[idx]) if i != idx]
# 유사도가 높은 순서대로 정렬
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
# 상위 5개의 인덱스와 유사도를 추출 
sim_scores[0:5]

[(1, 0.5914529785927046),
 (2, 0.5800116163408745),
 (3, 0.5648763295425474),
 (5, 0.5517234820715383),
 (4, 0.5409202661503921)]

In [145]:
key = input("찾고싶은 향수의 키워드를 입력하세요: ")
get_recommendations(str(key))

찾고싶은 향수의 키워드를 입력하세요: 페라가모 인칸토 참 EDT 30ml


1                 랑방 루머 2 로즈 30ml
2                지미추 블러썸 EDP 40ml
3                지미추 블러썸 EDP 60ml
5                  랑방 잔느 EDP 50ml
4    필로소피 어메이징그레이스 발레 로즈 EDT 60ml
Name: Name, dtype: object